<a href="https://colab.research.google.com/github/CaioPassos3/AprendizagemDeMaquina/blob/main/Lista2_AprendizagemDeMaquina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

Questão 1

Considere o conjunto de dados disponível em breastcancer.csv, organizado
em 31 colunas, sendo as 30 primeiras colunas os atributos e a última coluna a
saída. Os 30 atributos coletados de exames médicos são usados no diagnóstico
do câncer de mama, sendo 1 a classe positiva e 0 a classe negativa. Maiores
detalhes sobre os dados podem ser conferidos em https://scikit-learn.org/
stable/datasets/toy_dataset.html#breast-cancer-dataset

a) Considerando uma validação cruzada em 10 folds, avalie modelos de classificação
binária nos dados em questão. Para tanto, use as abordagens
abaixo:

– Regressão logística (treinado com GD ou SGD);

– Análise do discriminante Gaussiano;

– Naive Bayes Gaussiano;

In [2]:
df = pd.read_csv("breastcancer.csv")

# Separarando atributos e rótulos
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Validando cruzada com 10 folds
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Modelos com pipeline (normalização + classificador)
modelos = {
    "Regressão Logística (SGD)": make_pipeline(StandardScaler(), SGDClassifier(loss='log_loss', max_iter=1000, tol=1e-3, random_state=42)),
    "Análise do Discriminante Gaussiano (LDA)": make_pipeline(StandardScaler(), LinearDiscriminantAnalysis()),
    "Naive Bayes Gaussiano": make_pipeline(StandardScaler(), GaussianNB())
}

# Avaliando e imprimindo resultados
for nome, modelo in modelos.items():
    scores = cross_val_score(modelo, X, y, cv=cv, scoring='accuracy')
    print(f"\n{nome}")
    print(f"Acurácias por fold: {np.round(scores, 4)}")
    print(f"Média da acurácia: {scores.mean():.4f}")
    print(f"Desvio padrão: {scores.std():.4f}")


Regressão Logística (SGD)
Acurácias por fold: [0.9474 0.9649 0.9825 0.9825 0.9474 1.     0.9649 0.9298 0.9464 1.    ]
Média da acurácia: 0.9666
Desvio padrão: 0.0229

Análise do Discriminante Gaussiano (LDA)
Acurácias por fold: [0.9825 0.9474 0.9649 0.9474 0.9649 0.9825 0.9649 0.9298 0.9107 0.9643]
Média da acurácia: 0.9559
Desvio padrão: 0.0214

Naive Bayes Gaussiano
Acurácias por fold: [0.9474 0.9474 0.9474 0.9474 0.9474 0.9825 0.9123 0.8421 0.8929 0.9821]
Média da acurácia: 0.9349
Desvio padrão: 0.0402


b) Para cada modelo criado, reporte valor médio e desvio padrão da acurácia
global e da acurácia por classe.

In [7]:
df = pd.read_csv("breastcancer.csv")

# Separando atributos e rótulos
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Validando cruzada com 10 folds
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Modelos com pipeline
modelos = {
    "Regressão Logística (SGD)": make_pipeline(StandardScaler(), SGDClassifier(loss='log_loss', max_iter=1000, tol=1e-3, random_state=42)),
    "Análise do Discriminante Gaussiano (LDA)": make_pipeline(StandardScaler(), LinearDiscriminantAnalysis()),
    "Naive Bayes Gaussiano": make_pipeline(StandardScaler(), GaussianNB())
}

# Avaliação detalhada
resultados = {}

for nome, modelo in modelos.items():
    accs = []
    class_acc_0 = []
    class_acc_1 = []

    for train_idx, test_idx in cv.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)

        accs.append(accuracy_score(y_test, y_pred))

        # Acurácia por classe
        recall_0 = recall_score(y_test, y_pred, pos_label=0)
        recall_1 = recall_score(y_test, y_pred, pos_label=1)
        class_acc_0.append(recall_0)
        class_acc_1.append(recall_1)

    resultados[nome] = {
        "Acurácia Global Média": np.mean(accs),
        "Desvio Padrão Global": np.std(accs),
        "Acurácia Classe 0 Média": np.mean(class_acc_0),
        "Desvio Padrão Classe 0": np.std(class_acc_0),
        "Acurácia Classe 1 Média": np.mean(class_acc_1),
        "Desvio Padrão Classe 1": np.std(class_acc_1)
    }

resultados

{'Regressão Logística (SGD)': {'Acurácia Global Média': np.float64(0.9665726817042607),
  'Desvio Padrão Global': np.float64(0.022888141855148977),
  'Acurácia Classe 0 Média': np.float64(0.9692063492063492),
  'Desvio Padrão Classe 0': np.float64(0.031672136218575835),
  'Acurácia Classe 1 Média': np.float64(0.9621212121212122),
  'Desvio Padrão Classe 1': np.float64(0.035582861954055404)},
 'Análise do Discriminante Gaussiano (LDA)': {'Acurácia Global Média': np.float64(0.9559210526315789),
  'Desvio Padrão Global': np.float64(0.021428227913528535),
  'Acurácia Classe 0 Média': np.float64(0.9944444444444445),
  'Desvio Padrão Classe 0': np.float64(0.011111111111111117),
  'Acurácia Classe 1 Média': np.float64(0.8906926406926405),
  'Desvio Padrão Classe 1': np.float64(0.05678467368980479)},
 'Naive Bayes Gaussiano': {'Acurácia Global Média': np.float64(0.9348684210526315),
  'Desvio Padrão Global': np.float64(0.04020058377309513),
  'Acurácia Classe 0 Média': np.float64(0.95547619047

Questão 2

Considere o conjunto de dados disponível em vehicle.csv, organizado em 19 colunas,
sendo as 18 primeiras colunas os atributos e a última coluna a saída. Os 18
atributos caracterizam a silhueta de veículos, extraídos pelo método HIPS (Hierarchical
Image Processing System). A tarefa consiste em classificar o veículo
em 4 classes (bus, opel, saab, e van). Maiores detalhes sobre os dados podem
ser conferidos em https://www.openml.org/search?type=data&sort=runs&
id=54.

a) Considerando uma validação cruzada em 10 folds, avalie modelos de classificação
multiclasse nos dados em questão. Para tanto, use as abordagens
abaixo:

– Regressão softmax (treinado com GD ou SGD);

– Análise do discriminante Gaussiano;

– Naive Bayes Gaussiano;

In [10]:
data = pd.read_csv("vehicle.csv")

# Separando atributos e rótulos
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Criando 10-fold cross-validation
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Regressão Softmax (Logistic Regression multinomial, automático)
softmax_model = make_pipeline(StandardScaler(),
                              LogisticRegression(solver='saga', max_iter=5000, random_state=42))
softmax_scores = cross_val_score(softmax_model, X, y, cv=cv, scoring='accuracy')
print(f'Regressão Softmax - Acurácia média: {np.mean(softmax_scores):.4f} (+/- {np.std(softmax_scores):.4f})')

# Análise Discriminante Gaussiano (QDA)
gda_model = QuadraticDiscriminantAnalysis()
gda_scores = cross_val_score(gda_model, X, y, cv=cv, scoring='accuracy')
print(f'Análise Discriminante Gaussiano - Acurácia média: {np.mean(gda_scores):.4f} (+/- {np.std(gda_scores):.4f})')

# Naive Bayes Gaussiano
nb_model = GaussianNB()
nb_scores = cross_val_score(nb_model, X, y, cv=cv, scoring='accuracy')
print(f'Naive Bayes Gaussiano - Acurácia média: {np.mean(nb_scores):.4f} (+/- {np.std(nb_scores):.4f})')

Regressão Softmax - Acurácia média: 0.7965 (+/- 0.0366)
Análise Discriminante Gaussiano - Acurácia média: 0.8557 (+/- 0.0427)
Naive Bayes Gaussiano - Acurácia média: 0.4508 (+/- 0.0468)


b) Para cada modelo criado, reporte valor médio e desvio padrão da acurácia
global e da acurácia por classe.

In [13]:
data = pd.read_csv("vehicle.csv")
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Configurando validação cruzada
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Lista de modelos
models = {
    'Regressão Softmax': make_pipeline(StandardScaler(), LogisticRegression(solver='saga', max_iter=5000, random_state=42)),
    'Análise Discriminante Gaussiano': QuadraticDiscriminantAnalysis(),
    'Naive Bayes Gaussiano': GaussianNB()
}

# Armazenando resultados
results = {}

for name, model in models.items():
    accuracies = []
    fold_reports = []

    for train_idx, test_idx in cv.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Acurácia global
        acc = accuracy_score(y_test, y_pred)
        accuracies.append(acc)

        # Reporte por classe
        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
        fold_reports.append(report)

    # Calculando média/desvio global
    mean_acc = np.mean(accuracies)
    std_acc = np.std(accuracies)

    # Calculando média/desvio por classe
    classes = sorted(list(set(y)))
    class_means = {}
    class_stds = {}

    for cls in classes:
        cls_str = str(cls)  # Garantir que a chave seja string
        cls_accs = [fold_report[cls_str]['precision'] for fold_report in fold_reports if cls_str in fold_report]
        class_means[cls] = np.mean(cls_accs)
        class_stds[cls] = np.std(cls_accs)

    # Guardando resultados
    results[name] = {
        'mean_acc': mean_acc,
        'std_acc': std_acc,
        'class_means': class_means,
        'class_stds': class_stds
    }

# Exibindo resultados
for name, res in results.items():
    print(f'\n{name}')
    print(f'  Acurácia global média: {res["mean_acc"]:.4f}')
    print(f'  Acurácia global desvio padrão: {res["std_acc"]:.4f}')
    print('  Acurácia por classe (precisão média):')
    for cls in res['class_means']:
        print(f'    {cls} - média: {res["class_means"][cls]:.4f}, desvio: {res["class_stds"][cls]:.4f}')


Regressão Softmax
  Acurácia global média: 0.7965
  Acurácia global desvio padrão: 0.0366
  Acurácia por classe (precisão média):
    0.0 - média: 0.9124, desvio: 0.0725
    1.0 - média: 0.6829, desvio: 0.0952
    2.0 - média: 0.6705, desvio: 0.0544
    3.0 - média: 0.9278, desvio: 0.0597

Análise Discriminante Gaussiano
  Acurácia global média: 0.8557
  Acurácia global desvio padrão: 0.0427
  Acurácia por classe (precisão média):
    0.0 - média: 0.9865, desvio: 0.0206
    1.0 - média: 0.7453, desvio: 0.0762
    2.0 - média: 0.7765, desvio: 0.0839
    3.0 - média: 0.9101, desvio: 0.0569

Naive Bayes Gaussiano
  Acurácia global média: 0.4508
  Acurácia global desvio padrão: 0.0468
  Acurácia por classe (precisão média):
    0.0 - média: 0.6683, desvio: 0.2819
    1.0 - média: 0.4959, desvio: 0.1445
    2.0 - média: 0.5017, desvio: 0.0898
    3.0 - média: 0.3981, desvio: 0.0336
